In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [2]:
season_code = "25_26"
current_dir = Path.cwd()  
path_folder = current_dir.parent.parent.parent.parent / "csv" / f"csv{season_code}" / "players" / "centiles"
path_folder_end = current_dir.parent.parent.parent.parent / "csv" / f"csv{season_code}" / "players" / "metrics"

In [3]:
def line_index(df):
    weights = {
        "Goals Against": 10,
        "Saves": 10,
        "Clean Sheets": 5,
        "Defensive Actions Outside Penalty Area": 5,
        "Crosses Stopped": 5,
        "Save Efficiency": 60,
        "% Saves": 10,
        "% Crosses Stopped": 5,
        "PSxG/Save": 80
    }
    total_weights = sum(weights.values())
    return df[list(weights.keys())].multiply(pd.Series(weights), axis=1).sum(axis=1) / total_weights

In [4]:
def passes_index(df):
    weights = {
        "Completed Long Passes": 40,
        "Attempted Passes (excluding GK)": 20,
        "Attempted Throws": 20,
        "% Long Passes": 40,
    }
    total_weights = sum(weights.values())
    return df[list(weights.keys())].multiply(pd.Series(weights), axis=1).sum(axis=1) / total_weights

In [5]:
def create_indices_df(df_centiles):
    static_cols = ['Player', 'Nationality', 'Team', 'League', 'Position', 'General Position', 'Age', 'Minutes', 'Matches']
    
    df_indices = df_centiles[static_cols].copy()
    
    df_indices["Line Index"] = line_index(df_centiles).round().astype(int)
    df_indices["Passes Index"] = passes_index(df_centiles).round().astype(int)

    return df_indices

In [6]:
os.makedirs(path_folder_end, exist_ok=True)

for filename in os.listdir(path_folder):
    if filename.endswith("data_goals_centiles.csv"):
        path_file = os.path.join(path_folder, filename)
        
        path_end_indices = path_folder_end / "data_goals_metrics.csv"

        df_centiles = pd.read_csv(path_file)
        df_indices = create_indices_df(df_centiles)
        df_indices.to_csv(path_end_indices, index=False)

        print(f"Fichier traité et sauvegardé : {filename}")

Fichier traité et sauvegardé : data_goals_centiles.csv
